In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA

from scipy.integrate import solve_ivp
from scipy import integrate
from scipy.linalg import null_space

from scipy.optimize import fsolve
from scipy.integrate import solve_bvp

import pickle
import pandas as pd
import cmath

from itertools import chain

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
%matplotlib notebook

In [ ]:
# length
L = 1.7

# orbital number
l = 2

# coupling srength left/right
t_L = 0.7
t_R = t_L

# effective mass
mx = 9

Delta = 1

In [ ]:
# SOI and ZF

b_x = 0.2
b_y = 0.0
b_z = 0.2

gamma_SO = 0.2

In [ ]:
phi_0 = np.linspace(0, 2*np.pi, num=101)
phi_0[0] = 0.001
phi_0[-1] -= 0.001
phi_0[50] += 0.001

In [ ]:
lambda_coupl = 1 # 0.1 # Andreev-continuum coupling rate
damping_param = 0.1 # 0.01
Omega = 0.01

T_ferm = 0.5
T_bos = 0.2

alpha_0 = 0.001 # 0.0001
omega_c = 1

In [ ]:
def bose_distrib(x, T):
    """ Bose distribution function that handles large x/T to avoid overflow in the exponent """  

    y = x / T
    cutoff = 500.0  # cutoff to avoid overflow in exp
    
    if y >= 0:
        if y > cutoff:
            return 0.0  # for large y, n_B -> 0
        else:
            return 1.0 / (np.exp(y)-1)  
    else:
        y = -y
        if y > cutoff:
            return -1.0  # for large negative y, expression goes to -1 (since n_B -> 0)
        else:
            return -1.0 - 1.0 / (np.exp(y)-1)

In [ ]:
def fermi_distrib(x, T):
    """ Fermi distribution function that handles large x/T to avoid overflow in the exponent """    
    
    y = x / T
    cutoff = 500.0  # cutoff to avoid overflow in exp
    
    if y > cutoff:
        return 0.0  # for large positive y, 1/(exp(y)+1) -> 0
    elif y < -cutoff:
        return 1.0  # for large negative y, 1/(exp(y)+1) -> 1
    else:
        return 1.0 / (np.exp(y) + 1.0)

In [ ]:
# spectral density function

def spectral_density(x, lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    J = lambda_coupl**2 * damping_param * (1/((x - Omega)**2 + damping_param**2/4) - \
                                                   1/((x + Omega)**2 + damping_param**2/4))
    
    J_ohm = alpha_0 * x * np.exp(-np.abs(x)/omega_c)
    
    return J + J_ohm

### Finding free dot eigenenergies and eigenfunctions -  hopping parameters

In [ ]:
def chi_n_sigma(x, n, sigma):
    
    q = mx*gamma_SO
    k_n = n*np.pi/L
    
    cos_n = (k_n + sigma*q)/np.sqrt(2*(k_n**2 + q**2))
    sin_n = (k_n - sigma*q)/np.sqrt(2*(k_n**2 + q**2))
    
    f = np.exp(-1j*sigma*q*(x - L/2)) * (cos_n * np.exp(1j*k_n*(x-L/2)) + \
                                   sin_n * np.exp(-1j*k_n*(x-L/2)))
    
    return f/np.sqrt(L)

In [ ]:
x_plot = np.linspace(-L/2, L/2, 100)

plt.xlabel("x")
plt.ylabel("y")



plt.plot(x_plot, np.real(chi_n_sigma(x_plot, 3, -1)))
plt.plot(x_plot, np.imag(chi_n_sigma(x_plot, 3, -1)))

plt.grid()

In [ ]:
def cos_n_s(n, sigma):
    k_n = n*np.pi/L
    q = mx*gamma_SO
    
    cos_n = (k_n + sigma*q)/np.sqrt(2*(k_n**2 + q**2))
    
    return cos_n   

In [ ]:
def J_mn_sigma(m, n, sigma):
    
    q = mx*gamma_SO
    k_n = n*np.pi/L
    k_m = m*np.pi/L
    
    f_s_mn = cos_n_s(n, sigma) * cos_n_s(m, -sigma) / (k_n - k_m - 2*q*sigma) -\
             cos_n_s(n, -sigma) * cos_n_s(m, sigma) / (k_n - k_m + 2*q*sigma) +\
             cos_n_s(n, sigma) * cos_n_s(m, sigma) / (k_n + k_m - 2*q*sigma) -\
             cos_n_s(n, -sigma) * cos_n_s(m, -sigma) / (k_n + k_m + 2*q*sigma)
    
    if (-1)**(n+m) == 1:
        J = 2*b_x * f_s_mn / L * (-sigma)*np.sin(q*L)
    if (-1)**(n+m) == -1:
        J = 2*b_x * f_s_mn / L * (-1j)*np.cos(q*L)
    
    return J

In [ ]:
N = 50
n_arr = np.arange(1, N + 1)
q = mx*gamma_SO

k_n = n_arr * np.pi / L
eps_n = k_n**2 / (2 * mx) - mx*gamma_SO**2/2
m_one_arr = (-1)**n_arr

xi_n = np.sqrt(2/L) * k_n / np.sqrt(k_n**2 + (mx*gamma_SO)**2)

J_min = np.zeros((N, N), dtype='complex')
J_pl = np.zeros_like(J_min)

for n in range(N):
    for m in range(N):
        J_min[n, m] = J_mn_sigma(n+1, m+1, -1)
        J_pl[m, n] = np.conjugate(J_min[n, m])       

#### We consider transport through 2 dot levels - they get split intp 4 levels

In [ ]:
H_2 = np.zeros((2*N, 2*N), dtype='complex')

diagonal_elements = np.array([(eps + b_z, eps - b_z) for eps in eps_n]).flatten()
H_diag = np.diag(diagonal_elements)

for n in range(N):
    for m in range(N):
        H_2[2*n, 2*m+1] = J_min[n, m]
        H_2[2*n+1, 2*m] = J_pl[n, m]
        
H_2 += H_diag
E_2, Psi_2 = LA.eigh(H_2)
print(E_2[0:4])

for i in range(2*N):
    sum_i = np.real(np.sum(Psi_2[:, i]**2))
    Psi_2[:, i] /= sum_i
    
even_rows = Psi_2[::2, :]
u = np.sum(xi_n[:, np.newaxis] * even_rows, axis=0)
t_L_up = np.exp(1j*L*q) * np.sum(m_one_arr[:, np.newaxis] * xi_n[:, np.newaxis] * even_rows, axis=0)


odd_rows = Psi_2[1::2, :]
v = np.sum(xi_n[:, np.newaxis] * odd_rows, axis=0)
t_L_down = np.exp(-1j*L*q) * np.sum(m_one_arr[:, np.newaxis] * xi_n[:, np.newaxis] * odd_rows, axis=0)

In [ ]:
Gamma_L = np.zeros((4, 4), dtype='complex')
Gamma_R = np.zeros_like(Gamma_L)
F_R = np.zeros_like(Gamma_L)
F_L = np.zeros_like(Gamma_L)

for mu in range(4):
    for nu in range(4):
        Gamma_R[mu, nu] = np.conjugate(u[mu])*u[nu] + np.conjugate(v[mu])*v[nu]
        Gamma_L[mu, nu] = u[mu]*np.conjugate(u[nu]) + v[mu]*np.conjugate(v[nu])
    
        F_R[mu, nu] = u[mu]*v[nu] - u[nu]*v[mu]
        F_L[mu, nu] = np.conjugate(u[mu]*v[nu] - u[nu]*v[mu])  

Gamma_L *= t_L**2
Gamma_R *= t_L**2
F_L *= t_L**2
F_R *= t_L**2

### Finding Andreev states

In [ ]:
def G_inv(x, i):
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    
    Lambda = -1 / cmath.sqrt(Delta**2 - x**2) \
            * (x * np.block([[(Gamma_L + Gamma_R), zero_m], [zero_m, np.conjugate(Gamma_L + Gamma_R)]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * np.conjugate(np.transpose(F_L))], \
                                [np.exp(1j * s_L * phi_0[i]/2) * F_L, zero_m]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * np.conjugate(np.transpose(F_R))], \
                                [np.exp(1j * s_R * phi_0[i]/2) * F_R, zero_m]]))
    
    G_inv = x * np.eye(4*l) - np.block([[np.diag(E_2[0:4]), zero_m], [zero_m, -np.diag(E_2[0:4])]]) - Lambda
    
    return LA.det(G_inv)   

In [ ]:
roots = np.zeros((len(phi_0), 4*l))

for i in range(len(phi_0)):
    roots[i, 0] = fsolve(G_inv, 0.3, args=(i))  
    roots[i, 1] = fsolve(G_inv, 0.65, args=(i))  
    roots[i, 2] = fsolve(G_inv, 0.86, args=(i))  
    roots[i, 3] = fsolve(G_inv, 0.999, args=(i))
    
    roots[i, 4] = fsolve(G_inv, -0.3, args=(i))  
    roots[i, 5] = fsolve(G_inv, -0.65, args=(i))     
    roots[i, 6] = fsolve(G_inv, -0.86, args=(i))  
    roots[i, 7] = fsolve(G_inv, -0.999, args=(i))   
    
# # if needed adjust the initial guess for fsolve for a specific phi_0 region

# for i in range(20):
#     roots[i, 1] = fsolve(G_inv, 0.63, args=(i)) 
#     roots[-i, 1] = fsolve(G_inv, 0.57, args=(-i)) 
    
#     roots[i, 5] = fsolve(G_inv, -0.63, args=(i)) # 0.6
#     roots[-i, 5] = fsolve(G_inv, -0.57, args=(-i))     

In [ ]:
data = pd.DataFrame({
    "phi_0": phi_0/np.pi,
    "E_0": roots[:,0],
    "E_1": roots[:,1],
    "E_2": roots[:,2],
    "E_3": roots[:,3]
})

# Save the DataFrame to a text file (tab-separated)
data.to_csv("ABS_spin.txt", sep="\t", index=False)

In [ ]:
plt.plot(phi_0/np.pi, roots[:,0], linewidth=1.5)
plt.plot(phi_0/np.pi, roots[:,1], linewidth=1.5)
plt.plot(phi_0/np.pi, roots[:,2], linewidth=1.5)
plt.plot(phi_0/np.pi, roots[:,3], linewidth=1.5)

plt.title(r"$t_L={}, t_R={}, \gamma_{{SO}} = {},$"\
          .format(t_L, t_R, gamma_SO)+'\n'+
          r"$b_x={}\Delta, b_z = {}\Delta, L={}\Delta^{{-1}}$"\
          .format(np.round(b_x, 2), np.round(b_z, 2), L), fontsize=13)



plt.grid()
plt.xlabel(r'$\varphi$, $\pi$', fontsize=12)
plt.ylabel(r'$E_A$, $\Delta$', fontsize=12)


### AL to AL transition rates

In [ ]:
# diagonalizing the Hamiltonian: get eigenenergies - ABS, and eigenfunctions

def diagonal(x):
    
    eigval_arr = np.zeros((len(phi_0), 4*l))
    eigvec_arr = np.zeros((len(phi_0), 4*l, 4*l), dtype='complex')
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    for i in range(len(phi_0)):
    
        Lambda = -1 / np.sqrt(Delta**2 - x[i]**2) \
            * (x[i] * np.block([[(Gamma_L + Gamma_R), zero_m], [zero_m, (Gamma_L + Gamma_R)]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * np.conjugate(np.transpose(F_L))], \
                                [np.exp(1j * s_L * phi_0[i]/2) * F_L, zero_m]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * np.conjugate(np.transpose(F_R))], \
                                [np.exp(1j * s_R * phi_0[i]/2) * F_R, zero_m]]))
        
    
        H = np.block([[np.diag(E_2[0:4]), zero_m], [zero_m, -np.diag(E_2[0:4])]]) + Lambda
    
        
        eigval_arr[i], eigvec_arr[i] = LA.eigh(H)
    
    return eigval_arr, eigvec_arr 

In [ ]:
eigv_0, eigvec_0 = diagonal(roots[:, 0])
eigv_1, eigvec_1 = diagonal(roots[:, 1])
eigv_2, eigvec_2 = diagonal(roots[:, 2])
eigv_3, eigvec_3 = diagonal(roots[:, 3])

eigv_4, eigvec_4 = diagonal(roots[:, 4])
eigv_5, eigvec_5 = diagonal(roots[:, 5])
eigv_6, eigvec_6 = diagonal(roots[:, 6])
eigv_7, eigvec_7 = diagonal(roots[:, 7])

In [ ]:
plt.plot(phi_0/np.pi, eigv_0[:, 4])
plt.plot(phi_0/np.pi, eigv_1[:, 5])
plt.plot(phi_0/np.pi, eigv_2[:, 6])
plt.plot(phi_0/np.pi, eigv_3[:, 7])

plt.plot(phi_0/np.pi, eigv_4[:, 3])
plt.plot(phi_0/np.pi, eigv_5[:, 2])
plt.plot(phi_0/np.pi, eigv_6[:, 1])
plt.plot(phi_0/np.pi, eigv_7[:, 0])


plt.grid()

In [ ]:
# Eigenvectors corresponding to the Andreev energies

eta_0 = eigvec_0[:, :, 4]
eta_1 = eigvec_1[:, :, 5]
eta_2 = eigvec_2[:, :, 6]
eta_3 = eigvec_3[:, :, 7]

eta_4 = eigvec_4[:, :, 3]
eta_5 = eigvec_5[:, :, 2]
eta_6 = eigvec_6[:, :, 1]
eta_7 = eigvec_7[:, :, 0]

eta_arr = np.stack([eta_0, eta_1, eta_2, eta_3, eta_4, eta_5, eta_6, eta_7], axis=0)

In [ ]:
def curr_matr(x, i):
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    Lambda_L = x * np.block([[Gamma_L , zero_m], [zero_m, Gamma_L]]) \
             - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * np.conjugate(np.transpose(F_L))], \
                                [np.exp(1j * s_L * phi_0[i]/2) * F_L, zero_m]]) 
    
    Lambda_R = x * np.block([[Gamma_R , zero_m], [zero_m, Gamma_R]]) \
             - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * np.conjugate(np.transpose(F_R))], \
                                [np.exp(1j * s_R * phi_0[i]/2) * F_R, zero_m]])
        
    I = -1 / np.sqrt(Delta**2 - x**2) / (2*1j) * (s_L * Lambda_L + s_R * Lambda_R)
    
    return I

In [ ]:
def curr_nl(E_n, E_lamb, eta_n, eta_lamb, i):
        
    tau_z = np.block([[np.eye(4), np.zeros((4, 4))], [np.zeros((4, 4)), -np.eye(4)]])
    
    matr = tau_z @ curr_matr(E_lamb, i) - curr_matr(E_n, i) @ tau_z
    
    return np.conj(eta_n) @ matr @ eta_lamb    

In [ ]:
I_nl = np.zeros((len(phi_0), 4*l, 4*l), dtype='complex')
G_nl = np.zeros((len(phi_0), 4*l, 4*l))

for i in range(len(phi_0)):
    for n in range(4*l):
        for lamd in range(4*l):
            I_nl[i, n ,lamd] = curr_nl(roots[i, n], roots[i, lamd], eta_arr[n, i], eta_arr[lamd, i], i)

In [ ]:
plt.plot(phi_0/np.pi, np.abs(I_nl[:, 1, 0]))
plt.plot(phi_0/np.pi, np.abs(I_nl[:, 0, 1]))

# # Uncomment to plot different current matrix elements

# plt.plot(phi_0/np.pi, np.abs(I_nl[:, 3, 2]))
# plt.plot(phi_0/np.pi, np.abs(I_nl[:, 2, 3]))

In [ ]:
for i in range(len(phi_0)):
    for n in range(4*l):
        for lamd in range(4*l):
            
            energy_diff = roots[i, n] - roots[i, lamd]
            J_spect = spectral_density(energy_diff, lambda_coupl, alpha_0, damping_param, Omega, omega_c)
            bose = bose_distrib(energy_diff, T_bos)
            
            G_nl[i, lamd, n] = 2*np.pi * np.abs(I_nl[i, n, lamd])**2 * J_spect * bose 
            
            if math.isnan(G_nl[i, lamd, n]):
                G_nl[i, lamd, n] = 0

In [ ]:
G_AL_pp = G_nl[:, 0:4, 0:4]
G_AL_pm = G_nl[:, 0:4, 4:8]
G_AL_mp = G_nl[:, 4:8, 0:4]

In [ ]:
plt.plot(phi_0/np.pi, G_AL_pp[:, 0, 1], label=r'$\Gamma_{12}$')
plt.plot(phi_0/np.pi, G_AL_pp[:, 1, 0], label=r'$\Gamma_{21}$')

# # Uncomment to plot different transition rates

# plt.plot(phi_0/np.pi, G_AL_pm[:, 0, 2], label=r'$\Gamma_{1\bar1}$')
# plt.plot(phi_0/np.pi, G_AL_pm[:, 0, 3], label=r'$\Gamma_{1\bar2} = \Gamma_{2\bar1}$')
# plt.plot(phi_0/np.pi, G_AL_pm[:, 1, 3], label=r'$\Gamma_{2\bar2}$')

# plt.plot(phi_0/np.pi, G_AL_mp[:, 0, 2], label=r'$\Gamma_{\bar1 1}$')
# plt.plot(phi_0/np.pi, G_AL_mp[:, 0, 3], label=r'$\Gamma_{\bar1 2} = \Gamma_{\bar2 1}$')
# plt.plot(phi_0/np.pi, G_AL_mp[:, 1, 3], label=r'$\Gamma_{\bar2 2}$')


plt.legend()

### Finding transition rates ABS -> continuum: DOS

In [ ]:
# DOS matrix

def spectr(x, i):
    
    eigval_arr = np.zeros((8))
    eigvec_arr = np.zeros((8, 8), dtype='complex')
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    Lambda = 1 / np.sqrt(x**2 - Delta**2) \
        * (x * np.block([[(Gamma_L + Gamma_R), zero_m], [zero_m, (Gamma_L + Gamma_R)]]) \
        - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * np.conjugate(np.transpose(F_L))], \
                                [np.exp(1j * s_L * phi_0[i]/2) * F_L, zero_m]]) \
        - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * np.conjugate(np.transpose(F_R))], \
                                [np.exp(1j * s_R * phi_0[i]/2) * F_R, zero_m]]))

    G_R = LA.inv(x * np.eye(8) - np.block([[np.diag(E_2[0:4]), zero_m], [zero_m, -np.diag(E_2[0:4])]]) \
          + 1j * np.sign(x) * Lambda)

    G_A = np.transpose(np.conj(G_R))

    Spectr = 1j * (G_R - G_A)

    eigval_arr, eigvec_arr = LA.eigh(Spectr)
    
    return eigval_arr, eigvec_arr    

In [ ]:
eigv_c = np.zeros((len(phi_0), 8))
eigvect_c = np.zeros((len(phi_0), 8, 8), dtype='complex')

for i in range(len(phi_0)):
    eigv_c[i], eigvect_c[i] = spectr(-1.1, i)

In [ ]:
plt.plot(phi_0/np.pi, eigv_c[:, 0])
plt.plot(phi_0/np.pi, eigv_c[:, 1])
plt.plot(phi_0/np.pi, eigv_c[:, 2])
plt.plot(phi_0/np.pi, eigv_c[:, 3])

plt.plot(phi_0/np.pi, eigv_c[:, 4])
plt.plot(phi_0/np.pi, eigv_c[:, 5])
plt.plot(phi_0/np.pi, eigv_c[:, 6])
plt.plot(phi_0/np.pi, eigv_c[:, 7])

In [ ]:
# current matrix for continuum states E > Delta - retarded square root in the denominator

def curr_matr_cont(x, i):
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    Lambda_L = x * np.block([[Gamma_L , zero_m], [zero_m, Gamma_L]]) \
             - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * np.conjugate(np.transpose(F_L))], \
                                [np.exp(1j * s_L * phi_0[i]/2) * F_L, zero_m]]) 
    
    Lambda_R = x * np.block([[Gamma_R , zero_m], [zero_m, Gamma_R]]) \
         - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * np.conjugate(np.transpose(F_R))], \
                                [np.exp(1j * s_R * phi_0[i]/2) * F_R, zero_m]])
    
    I = -1 / (-1j * np.sign(x) * np.sqrt(x**2 - Delta**2)) / (2*1j) \
        * (s_L * Lambda_L + s_R * Lambda_R)
    
    return I

In [ ]:
# continuum current matrix element

def curr_cont_nl(E, E_lamd, psi_n, eta_lamd, i):
    
    tau_z = np.block([[np.eye(4), np.zeros((4, 4))], [np.zeros((4, 4)), -np.eye(4)]])
    
    matr = tau_z @ curr_matr(E_lamd, i) - curr_matr_cont(E, i) @ tau_z
    
    return np.conj(psi_n) @ matr @ eta_lamd    

In [ ]:
def integr_gamma_out(x, E_lamd, eta_lamd, i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    E = x + E_lamd
    
    # 8 eigenvalues, eigenvectors for continuum state
    eigv_E, eigvect_E = spectr(E, i)
    
    # current elements
    I_nl = np.zeros((8), dtype='complex')
    
    for n in range(8): 
        I_nl = curr_cont_nl(E, E_lamd, eigvect_E[:, n], eta_lamd, i)
        
    sum_n = np.sum(eigv_E * np.abs(I_nl)**2)
    
    n_F = fermi_distrib(E, T_ferm)
    n_B = bose_distrib(x, T_bos)
    J_sp = spectral_density(x, lambda_coupl, alpha_0, damping_param, Omega, omega_c)
    
    return J_sp * n_B * (1 - n_F) * sum_n 
    

In [ ]:
def integr_gamma_in(x, E_lamd, eta_lamd, i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    E = x + E_lamd
    
    # 8 eigenvalues, eigenvectors for continuum state
    eigv_E, eigvect_E = spectr(E, i)
    
    # current elements
    I_nl = np.zeros((8), dtype='complex')
    
    for n in range(8): 
        I_nl = curr_cont_nl(E, E_lamd, eigvect_E[:, n], eta_lamd, i)
        
    sum_n = np.sum(eigv_E * np.abs(I_nl)**2)
    
    n_F = fermi_distrib(E, T_ferm)
    n_B = bose_distrib(x, T_bos)
    J_sp = spectral_density(x, lambda_coupl, alpha_0, damping_param, Omega, omega_c)
    
    return J_sp * (n_B + 1) * n_F * sum_n 
    

In [ ]:
def compute_integrals(i, lamd, Delta, roots, eta_arr, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    Gamma_out_pc = integrate.quad(integr_gamma_out, Delta - roots[i, lamd], np.infty,
                                  args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    Gamma_out_mc = integrate.quad(integr_gamma_out, -np.infty, -Delta - roots[i, lamd],
                                  args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    Gamma_in_pc = integrate.quad(integr_gamma_in, Delta - roots[i, lamd], np.infty,
                                 args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    Gamma_in_mc = integrate.quad(integr_gamma_in, -np.infty, -Delta - roots[i, lamd],
                                 args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    
    # Return results as a tuple
    return (i, lamd, Gamma_out_pc, Gamma_out_mc, Gamma_in_pc, Gamma_in_mc)

In [ ]:
Gamma_out_pc = np.zeros((len(phi_0), 4))
Gamma_out_mc = np.zeros_like(Gamma_out_pc) 
Gamma_in_pc = np.zeros_like(Gamma_out_pc)
Gamma_in_mc = np.zeros_like(Gamma_out_pc)

# Use ProcessPoolExecutor for parallel execution
with ProcessPoolExecutor() as executor:
    
    # Create a list of futures by submitting tasks for parallel execution
    futures = [executor.submit(compute_integrals, i, lamd, Delta, roots, eta_arr, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c)
               for i in range(len(phi_0)) for lamd in range(4)]
    
    # Collect results as they are completed
    for future in futures:
        i, lamd, out_pc, out_mc, in_pc, in_mc = future.result()
        
        # Store results in the appropriate arrays
        Gamma_out_pc[i, lamd] = out_pc
        Gamma_out_mc[i, lamd] = out_mc
        Gamma_in_pc[i, lamd] = in_pc
        Gamma_in_mc[i, lamd] = in_mc

In [ ]:
Gamma_in = (Gamma_in_pc + Gamma_in_mc) * 2*np.pi
Gamma_out = (Gamma_out_pc + Gamma_out_mc) * 2*np.pi

In [ ]:
plt.plot(phi_0/np.pi, Gamma_out[:, 0], label=r'$\Gamma^{out}_1$')
plt.plot(phi_0/np.pi, Gamma_in[:, 0], label=r'$\Gamma^{in}_1$')

plt.plot(phi_0/np.pi, Gamma_out[:, 1], label=r'$\Gamma^{out}_{2}$')
plt.plot(phi_0/np.pi, Gamma_in[:, 1], label=r'$\Gamma^{in}_{2}$')

plt.legend()
plt.grid()

## Lindblad euqation: steady state

In [ ]:
N = 4
N_state = 2**N

In [ ]:
# function that returns the momentum state (as string) for printing

def print_state(n):
    
    if len(bin(n)[:1:-1]) < N:
        new_bin = bin(n)[:1:-1]
        for i in range(N - len(bin(n)[:1:-1])):
            new_bin += '0'
            
        return new_bin
    else:
        return bin(n)[:1:-1]

In [ ]:
# the momentum vectors < q_i0 ... q_iNk | rho | q_j0 ... q_jNk >, where q's can be either 0 or 1 
# in case of fermions - by writing each i and j of the density matrix in the binary representation
# we get all possible combinations of occupied/non-occupied Nk momentum states

# this function finds whether the q'_iq entry (q = [0, Nk-1] index of the entry in the vector) is occupied or not 
def n(q, i):
    if q > len(bin(i)[:1:-1])-1:
        return 0
    else:
        return int(bin(i)[:1:-1][q])

In [ ]:
lvl = np.array(range(N))
states = np.array(range(N_state))

n_v = np.vectorize(n)

nf_h = np.array([n_v(lvl, si) for si in states])

In [ ]:
# steady state matrix
Steady_M = np.zeros((len(phi_0), N_state, N_state), dtype='float64') 

for j in range(len(phi_0)):
    
    # loop over the raws - as many as there are states - 16
    for i in range(N_state):

        # loop over 4 levels
        for mu in range(4):

            n_mu = nf_h[i, mu]

            # G_in
            Steady_M[j, i, i + (-1)**n_mu * 2**mu] += n_mu * (Gamma_in[j, mu] )

            # G_out
            Steady_M[j, i, i + (-1)**n_mu * 2**mu] += (1-n_mu) * (Gamma_out[j, mu] )

            # G_in, G_out
            Steady_M[j, i, i] -= n_mu * (Gamma_out[j, mu]) + \
                                (1-n_mu) * (Gamma_in[j, mu] )

    #             loop over lambda' != lamda
            for nu in chain(range(mu), range(mu+1, 4)):

                n_nu = nf_h[i, nu]

                Steady_M[j, i, i + (-1)**n_mu * 2**mu + (-1)**n_nu * 2**nu] += \
                                n_mu * (1-n_nu) * G_AL_pp[j, nu, mu] + \
                                n_mu * n_nu * G_AL_mp[j, nu, mu] + \
                                (1-n_mu) * (1-n_nu) * G_AL_pm[j, nu, mu]

                Steady_M[j, i, i] -= (1-n_mu) * n_nu * G_AL_pp[j, nu, mu] + \
                                         (1-n_mu) * (1-n_nu) * G_AL_mp[j, nu, mu] +\
                                         n_mu * n_nu * G_AL_pm[j, nu, mu]


In [ ]:
ns = np.zeros((len(phi_0), 16), dtype='float64')

for i in range(len(phi_0)):
    ns[i] = null_space(Steady_M[i], rcond=10e-16)[:,0]
#     print(i, LA.matrix_rank(Steady_M[i]))

steady_occup = ns
steady_occup /= np.sum(steady_occup, axis=1, keepdims=True)

In [ ]:
for i in range(N_state):
    print(nf_h[i], '\t', steady_occup[51, i])

In [ ]:
plt.plot(phi_0/np.pi, steady_occup[:, 0], label=r'$P_0$')
plt.plot(phi_0/np.pi, steady_occup[:, 1], label=r'$P_1$')
plt.plot(phi_0/np.pi, steady_occup[:, 2], label=r'$P_2$')
plt.plot(phi_0/np.pi, steady_occup[:, 3], label=r'$P_3$')
plt.plot(phi_0/np.pi, steady_occup[:, 4], label=r'$P_4$')
plt.plot(phi_0/np.pi, steady_occup[:, 5], label=r'$P_5$')
plt.plot(phi_0/np.pi, steady_occup[:, 6], label=r'$P_6$')

plt.grid()
plt.legend()

### Relaxation time calculation

In [ ]:
def rhs1(t, P, i_eq):

    # find derivatives of N_state elements
    dP = np.zeros(N_state)
        
    dP = Steady_M[i_eq] @ P
        
    return dP

In [ ]:
def compute_tau(i1, i2):
    
    P0 = steady_occup[i1]
    sol_ = solve_ivp(rhs1, (0.0, 1e13), P0, method='LSODA', dense_output=True, args=(i2,))
    ts_ = sol_.t
    ys_ = np.stack(sol_.y)
    
    tau = 1e10  # Default value for tau
    for j in range(len(ts_)):
        if np.sum(np.abs(ys_[:, j] - steady_occup[i2, :])) < 1e-3:
            tau = ts_[j]
            break
            
    return i1, i2, tau

In [ ]:
# calculate relaxation time
tau_arr = np.zeros((len(phi_0), len(phi_0)))

# Parallel execution
with ProcessPoolExecutor() as executor:
    # Submit tasks for all (i1, i2) pairs
    futures = [executor.submit(compute_tau, i1, i2) for i1 in range(len(phi_0)) for i2 in range(len(phi_0))]
    
    # Collect results and populate tau_arr
    for future in futures:
        i1, i2, tau = future.result()
        tau_arr[i1, i2] = tau

# Now tau_arr contains the computed tau values

In [ ]:
plt.imshow(tau_arr[:, :], extent=[phi_0[0]/np.pi, phi_0[-1]/np.pi,\
                                         phi_0[0]/np.pi, phi_0[-1]/np.pi],\
           origin='lower', aspect='auto', cmap='turbo',)

plt.colorbar(label=r'$\tau, \Delta$')
plt.xlabel(r'$\varphi_{final}, \pi$')
plt.ylabel(r'$\varphi_{init}, \pi$')

plt.title(r"$L = {}\Delta^{{-1}}, \eta_d = {}, \Omega={}\Delta,$"\
          .format(L, damping_param, Omega)+'\n'+
          r"$T_{{qp}} = {}\Delta, T_b = {}\Delta, \gamma_{{SO}}={}, b_x = {}, b_z = {}$"\
          .format(T_ferm, T_bos, gamma_SO, b_x, b_z), fontsize=13)

# # uncomment to plot contour lines

# contour_levels = np.linspace(np.min(init_trace_dist[:, :]), 
#                              np.max(init_trace_dist[:, :]), 10)
# contour = plt.contour(init_trace_dist[:, :],
#                       levels=contour_levels,
#                       extent=[phi_0[0]/np.pi, phi_0[-1]/np.pi,
#                               phi_0[0]/np.pi, phi_0[-1]/np.pi],
#                       colors='black', linewidths=0.5)

plt.show()

In [ ]:
# calculate and plot initial trace distance

P0 = steady_occup[:, np.newaxis]
init_trace_dist = np.sum(np.abs(P0 - steady_occup), axis=2)

plt.imshow(init_trace_dist[:, :], extent=[phi_0[0]/np.pi, phi_0[-1]/np.pi,\
                                                   phi_0[0]/np.pi, phi_0[-1]/np.pi],
           origin='lower', aspect='auto', cmap='turbo', vmin=0)
plt.title(r"$L = {}\Delta^{{-1}}, \eta_d = {}, $"\
          .format(L,  damping_param)+'\n'+
          r"$\Omega={}\Delta, T_{{qp}}^{{-1}} = {}\Delta, T_b^{{-1}} = {}\Delta$"\
          .format(Omega, np.round(1/T_ferm, 2), np.round(1/T_bos, 2)), fontsize=13)

cbar = plt.colorbar()
cbar.set_label(r'$\mathcal{D}_T(0)$')  # Color bar label in log scale


plt.xlabel(r'$\varphi_{final}, \pi$')
plt.ylabel(r'$\varphi_{init}, \pi$')

plt.show()

In [ ]:
# write the initial/final phase, relaxation time tau and initial trace distance in a file

with open("tau_spin_1.8pi.txt", "w") as file:
    # write the header
    file.write("phi_init phi_f tau D0\n")
    
    # iterate over the array indices and write the data
    for i in range(len(phi_0)):
        for j in range(len(phi_0)):
            file.write(f"{phi_0[i]} {phi_0[j]} {tau_arr[i, j]} {init_trace_dist[i, j]}\n")